In [1]:
import os
import json
import io
from data_generator import DataGenerator
from trade_env import TraderEnv
from blog_ai import A3CAgent
from async_rl import *
import time
import numpy as np
from multiprocessing import *
from collections import deque

/Users/samirantonio/anaconda/envs/python3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
Using TensorFlow backend.


In [ ]:
stage_length_var = 10
def get_enviroment():
    return TraderEnv(DataGenerator(), stage_history_length=stage_length_var)

In [ ]:
manager = Manager()
weight_dict = manager.dict()
mem_queue = manager.Queue(256)

threads = 16

pool = Pool(threads + 1, init_worker)

for i in range(threads):
    pool.apply_async(generate_experience_proc, (mem_queue, weight_dict, i, get_enviroment))

pool.apply_async(learn_proc, (mem_queue, weight_dict, get_enviroment))

pool.close()
pool.join()


 30453> Process started
 30454> Process started
 30456> Process started
 30455> Process started
 30457> Process started
 30458> Process started
 30459> Process started
 30460> Process started
 30462> Process started
 30463> Process started
 30461> Process started
 30467> Process started
 30466> Process started
 30464> Process started
 30465> Process started
 30468> Process started
learn_proc
 30469> Learning process
 30469> Setting weights in dict
 30468> Loaded weights from dict
 30466> Loaded weights from dict
 30465> Loaded weights from dict
 30454> Loaded weights from dict
 30455> Loaded weights from dict
 30460> Loaded weights from dict
 30459> Loaded weights from dict
 30453> Loaded weights from dict
 30461> Loaded weights from dict
 30457> Loaded weights from dict
 30456> Loaded weights from dict
 30462> Loaded weights from dict
 30463> Loaded weights from dict
 30467> Loaded weights from dict
 30458> Loaded weights from dict
 30464> Loaded weights from dict
 30460> Best: 1367; 